In [ ]:
import polars as pl
import polars_ds as pld
import numpy as np

In [ ]:
df.select(
    pl.col("a").str2.sorensen_dice(pl.col("b"))
)

In [ ]:
df = pld.random_data(15_000)
df.head()

In [ ]:
import scipy

In [ ]:
df.select(pl.col("feature_1").num.rfft())

In [ ]:
df.select(pl.col("feature_1").num.rfft2())